# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 5.2.2
- **Date :** 11/11/2023
- **Heure :** 12:20
- **Données :** `ready_Cloud9am_Cloud3pm.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/alex/ready_Cloud9am_Cloud3pm.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum,NonMesNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-07-01,8.8,15.7,5.0,1.6,2.6,48.0,13.0,15.0,92.0,67.0,...,7,1,32,2.356194,3.926991,3.141593,-34.928181,138.599931,4,6
2008-07-02,12.7,15.8,0.8,1.4,7.8,35.0,13.0,15.0,75.0,52.0,...,7,2,32,3.926991,4.319690,3.926991,-34.928181,138.599931,4,6
2008-07-03,6.2,15.1,0.0,1.8,2.1,20.0,2.0,11.0,81.0,56.0,...,7,3,32,3.141593,1.178097,3.926991,-34.928181,138.599931,4,6
2008-07-04,5.3,15.9,0.0,1.4,8.0,30.0,6.0,13.0,71.0,46.0,...,7,4,32,1.178097,1.178097,0.785398,-34.928181,138.599931,4,6
2008-07-07,7.6,11.2,16.2,4.6,1.1,46.0,17.0,13.0,83.0,88.0,...,7,7,32,3.534292,2.748894,3.926991,-34.928181,138.599931,4,6


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5959 entries, 2008-07-01 to 2017-06-25
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         5959 non-null   float64
 1   MaxTemp         5959 non-null   float64
 2   Rainfall        5959 non-null   float64
 3   Evaporation     5959 non-null   float64
 4   Sunshine        5959 non-null   float64
 5   WindGustSpeed   5959 non-null   float64
 6   WindSpeed9am    5959 non-null   float64
 7   WindSpeed3pm    5959 non-null   float64
 8   Humidity9am     5959 non-null   float64
 9   Humidity3pm     5959 non-null   float64
 10  Pressure9am     5959 non-null   float64
 11  Pressure3pm     5959 non-null   float64
 12  Temp9am         5959 non-null   float64
 13  Temp3pm         5959 non-null   float64
 14  RainToday       5959 non-null   int64  
 15  RainTomorrow    5959 non-null   int64  
 16  year            5959 non-null   int64  
 17  month           5959 no

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[ 1, 24],
         [21, 24],
         [20, 23],
         [13, 20],
         [ 3, 22]]),
  array([[11, 21],
         [15, 23],
         [10, 13],
         [ 8, 17],
         [10, 13]]),
  array([[ 5, 14],
         [ 7, 21],
         [12, 15],
         [ 9, 18],
         [ 4,  9]]),
  array([[14, 24],
         [12, 15],
         [18, 21],
         [ 1, 21],
         [ 6, 16]]),
  array([[ 5, 23],
         [16, 23],
         [ 5, 22],
         [16, 23],
         [11, 24]]),
  array([[ 2, 17],
         [ 1, 15],
         [14, 17],
         [11, 15],
         [15, 19]]),
  array([[16, 20],
         [17, 23],
         [20, 23],
         [ 8, 22],
         [11, 14]]),
  array([[ 9, 22],
         [ 7, 22],
         [10, 23],
         [ 1, 11],
         [ 1, 19]]),
  array([[11, 23],
         [17, 23],
         [16, 19],
         [14, 19],
         [12, 15]]),
  array([[ 4, 13],
         [13, 23],
         [ 1,  8],
         [22, 25],
         [ 

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,696,29
1,128,140


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.96      0.90       725
           1       0.83      0.52      0.64       268

    accuracy                           0.84       993
   macro avg       0.84      0.74      0.77       993
weighted avg       0.84      0.84      0.83       993

